In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [7]:
pseudo = pd.read_csv("../input/tps-aug-2021-lgbm-xgb-catboost/submission.csv")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

# Optuna

In [8]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [9]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[scale_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                    'objective': 'regression',
              'metric':'rmse',
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 0.008599, 0.010599),
               'subsample': trial.suggest_uniform('subsample', 0.538, 0.568),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.58, 0.61),
                'subsample_freq': trial.suggest_int('subsample_freq', 2, 4),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 2.6, 2.8),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 4.92, 5.32),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 346, 358),
                 'min_child_samples':trial.suggest_int('min_child_samples', 6, 8),
          'importance_type': 'gain'
             # 'num_leaves':trial.suggest_int('num_leaves', 10, 200),
              #'max_depth':trial.suggest_int('max_depth', 5, 50),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
              # 'n_jobs' : -1,
                #'max_bin':trial.suggest_int('max_bin', 300, 1000),
              #'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              #'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[scale_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[scale_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-21 13:44:14,770] A new study created in memory with name: no-name-5799719f-0683-4efc-8ca4-65a1d49cb2a6


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77043
[2000]	valid_0's rmse: 7.75917
[3000]	valid_0's rmse: 7.75497
[4000]	valid_0's rmse: 7.7522
[5000]	valid_0's rmse: 7.75075
[6000]	valid_0's rmse: 7.7497
Early stopping, best iteration is:
[6269]	valid_0's rmse: 7.74944
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77116
[2000]	valid_0's rmse: 7.75949
[3000]	valid_0's rmse: 7.75511
[4000]	valid_0's rmse: 7.7528
[5000]	valid_0's rmse: 7.75128
Early stopping, best iteration is:
[5700]	valid_0's rmse: 7.74997


[I 2021-08-21 13:53:11,629] Trial 0 finished with value: 7.748196039675071 and parameters: {'learning_rate': 0.01042317371433061, 'subsample': 0.5578423448253642, 'colsample_bytree': 0.6092268400425587, 'subsample_freq': 2, 'reg_lambda': 2.773524452534491, 'reg_alpha': 4.957414541010295, 'min_child_weight': 350.7509252088729, 'min_child_samples': 6}. Best is trial 0 with value: 7.748196039675071.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77083
[2000]	valid_0's rmse: 7.75929
[3000]	valid_0's rmse: 7.756
[4000]	valid_0's rmse: 7.75352
[5000]	valid_0's rmse: 7.75204
[6000]	valid_0's rmse: 7.75071
Early stopping, best iteration is:
[6566]	valid_0's rmse: 7.75009
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77079
[2000]	valid_0's rmse: 7.75988
[3000]	valid_0's rmse: 7.75635
[4000]	valid_0's rmse: 7.75409
[5000]	valid_0's rmse: 7.75272
Early stopping, best iteration is:
[5654]	valid_0's rmse: 7.75188


[I 2021-08-21 14:02:05,840] Trial 1 finished with value: 7.749377862140794 and parameters: {'learning_rate': 0.01037682158461626, 'subsample': 0.5483295056625028, 'colsample_bytree': 0.6014595147346742, 'subsample_freq': 3, 'reg_lambda': 2.782493598133209, 'reg_alpha': 5.203628867486934, 'min_child_weight': 346.4201736943349, 'min_child_samples': 8}. Best is trial 0 with value: 7.748196039675071.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77091
[2000]	valid_0's rmse: 7.75961
[3000]	valid_0's rmse: 7.75649
[4000]	valid_0's rmse: 7.75445
Early stopping, best iteration is:
[4235]	valid_0's rmse: 7.75358
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77129
[2000]	valid_0's rmse: 7.75971
[3000]	valid_0's rmse: 7.7556
[4000]	valid_0's rmse: 7.75316
[5000]	valid_0's rmse: 7.75204
Early stopping, best iteration is:
[5644]	valid_0's rmse: 7.75065


[I 2021-08-21 17:44:27,247] Trial 28 finished with value: 7.75066515674935 and parameters: {'learning_rate': 0.010445944532131457, 'subsample': 0.5424775395651212, 'colsample_bytree': 0.5945409622233735, 'subsample_freq': 3, 'reg_lambda': 2.765384017726487, 'reg_alpha': 5.286350514403525, 'min_child_weight': 354.56442517972846, 'min_child_samples': 7}. Best is trial 0 with value: 7.748196039675071.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77234
[2000]	valid_0's rmse: 7.7609
[3000]	valid_0's rmse: 7.75694
[4000]	valid_0's rmse: 7.75459
[5000]	valid_0's rmse: 7.75232
[6000]	valid_0's rmse: 7.75099
Early stopping, best iteration is:
[6567]	valid_0's rmse: 7.75037
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 7.77302
[2000]	valid_0's rmse: 7.7611
[3000]	valid_0's rmse: 7.75692
[4000]	valid_0's rmse: 7.75453
Early stopping, best iteration is:
[4349]	valid_0's rmse: 7.75408


[I 2021-08-21 17:52:05,438] Trial 29 finished with value: 7.750774791213117 and parameters: {'learning_rate': 0.009718212524917369, 'subsample': 0.54608746752471, 'colsample_bytree': 0.5849262181277749, 'subsample_freq': 3, 'reg_lambda': 2.7843539501271852, 'reg_alpha': 5.22560454002953, 'min_child_weight': 356.03177603996573, 'min_child_samples': 6}. Best is trial 0 with value: 7.748196039675071.


Number of finished trials: 30
Best trial: {'learning_rate': 0.01042317371433061, 'subsample': 0.5578423448253642, 'colsample_bytree': 0.6092268400425587, 'subsample_freq': 2, 'reg_lambda': 2.773524452534491, 'reg_alpha': 4.957414541010295, 'min_child_weight': 350.7509252088729, 'min_child_samples': 6}


In [11]:
study.best_params

{'learning_rate': 0.01042317371433061,
 'subsample': 0.5578423448253642,
 'colsample_bytree': 0.6092268400425587,
 'subsample_freq': 2,
 'reg_lambda': 2.773524452534491,
 'reg_alpha': 4.957414541010295,
 'min_child_weight': 350.7509252088729,
 'min_child_samples': 6}

# Log

7.761521447852026 row-wise noise ver3

7.7627473037692365 random noise ver4

7.749394595681444 no noise ver6

7.7488157884604965 no noise ver8 (narrow space)

7.748628607085202 no noise ver10 (narrow space)

7.7483594115082495 no noise ver11 (narrow space)